# Files erkunden

Vor der eigentlichen Datenextraktion ist es entscheidend, die **Dateinamenstruktur** der JSON-Files zu analysieren.  
Die Filenames enthalten wichtige Informationen, die zur korrekten Aufteilung und Zuordnung der Daten verwendet werden.

Beispielhafte Struktur eines Dateinamens:
activity-2024-07-15-dc31ee69-703b-48b6-a4ec-d4bfd26452be.json

In [81]:
from pathlib import Path

data_dir = Path("/Users/frederickurbel/Downloads/polar-user-data-export_7e32ae84-89cf-4c56-b29e-d089feef4207")
print(data_dir)

/Users/frederickurbel/Downloads/polar-user-data-export_7e32ae84-89cf-4c56-b29e-d089feef4207


In [82]:
# json_files = list(data_dir.glob("*.json"))
# print(f"{len(json_files)} JSON-Dateien gefunden:")
# for f in json_files:
#     print(" -", f.name)

# Daten extrahieren – Prozess

Der Prozess zur Extraktion und Aufbereitung der Daten aus den JSON-Files umfasst folgende Schritte:

1. **Filtern der JSON-Files**  
   Trennung und Klassifizierung der Dateien in die Gruppen  
   `activity`, `training` und `247ohr` basierend auf dem jeweiligen Dateinamen.

2. **Zusammenstellen der DataFrames**  
   Einlesen der Dateien, Zusammenführen der Inhalte und Erstellen eines DataFrames pro Kategorie.

3. **Erkunden der DataFrames**  
   Erste Analyse mit Befehlen wie `.head()`, `.value_counts()`, `.info()` und `.shape()`,  
   um Struktur, Datentypen und Datenumfang zu prüfen.

4. **Löschen irrelevanter Features**  
   Entfernen redundanter oder nicht benötigter Spalten, um den Fokus auf relevante Informationen zu legen.

5. **Erkunden verschachtelter Features**  
   Untersuchung von Spalten, die Listen, Dictionaries oder lange JSON-artige Strings enthalten.

6. **Parsen und Feature Engineering**  
   Umwandlung verschachtelter Strukturen in einzelne Spalten (Parsing),  
   Extraktion relevanter Informationen und anschließendes Entfernen der ursprünglichen Rohspalten.

7. **Speichern der bereinigten DataFrames**  
   Export der finalen, sauberen DataFrames als `.csv`-Dateien für die anschließende Analyse (EDA).

## Activitätsdaten extrahieren

In [87]:
import json
import pandas as pd
from pathlib import Path

# Alle activity-*.json Dateien filtern
activity_files = [f for f in data_dir.glob("*.json") if f.name.startswith("activity-")]
print(f"{len(activity_files)} Activity-Dateien gefunden")

# Daten einlesen
activity_rows = []
for file in activity_files:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        
    # Metadata aus Dateiname extrahieren (Datum)
    date_str = file.name.split("-")[1]  # z.B. activity-2024-01-26
    activity_rows.append({"filename": file.name, "date": date_str, **data})

df_activity = pd.DataFrame(activity_rows)

# Datum formatieren
df_activity["date"] = pd.to_datetime(df_activity["date"], errors="coerce")

print(df_activity.info())
print(df_activity.head(3))

3087 Activity-Dateien gefunden
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3087 entries, 0 to 3086
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   filename             3087 non-null   object        
 1   date                 3087 non-null   datetime64[ns]
 2   exportVersion        3087 non-null   object        
 3   summary              1716 non-null   object        
 4   samples              3087 non-null   object        
 5   physicalInformation  3087 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 144.8+ KB
None
                                            filename       date exportVersion  \
0  activity-2024-01-26-320cd5c7-b056-4234-b0de-7a... 2024-01-26           2.6   
1  activity-2015-11-27-275dbb71-b116-409e-90bc-a6... 2015-11-27           2.6   
2  activity-2025-06-21-0d786ff5-3f65-44cc-9c18-8f... 2025-06-21           2.6   

                

In [ ]:
df_activity = df_activity.drop('exportVersion', axis=1)
df_activity = df_activity.drop('filename', axis=1)

In [85]:
df_activity['summary'][2]

{'startTime': '00:00:00.000',
 'endTime': '23:59:59.000',
 'stepCount': 25568,
 'stepsDistance': 21580.40234375,
 'calories': 3181,
 'sleepQuality': 0.0,
 'sleepDuration': 'PT0S',
 'inactivityAlertCount': 0,
 'dailyMetMinutes': 915.2999999999984,
 'activityLevels': [{'level': 'NO_DATA', 'duration': 'PT0S'},
  {'level': 'SLEEP', 'duration': 'PT7110S'},
  {'level': 'SEDENTARY', 'duration': 'PT10140S'},
  {'level': 'LIGHT', 'duration': 'PT16230S'},
  {'level': 'CONTINUOS_MODERATE', 'duration': 'PT3480S'},
  {'level': 'INTERMITTENT_MODERATE', 'duration': 'PT0S'},
  {'level': 'CONTINUOS_VIGOROUS', 'duration': 'PT9450S'},
  {'level': 'INTERMITTENT_VIGOROUS', 'duration': 'PT0S'},
  {'level': 'NON_WEAR', 'duration': 'PT39990S'}]}

In [89]:
import ast

def parse_summary(val):
    if isinstance(val, dict):
        return val
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except Exception:
            return {}
    return {}

def clean_time(s):
    return s.split(".")[0] if isinstance(s, str) else None

def pt_to_hms(s):
    # z. B. 'PT7110S' → '01:58:30'
    if not isinstance(s, str) or not s.startswith("PT"):
        return None
    secs = int(float(s[2:-1]))
    h, rem = divmod(secs, 3600)
    m, s = divmod(rem, 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

summary = df_activity["summary"].apply(parse_summary)

df_activity["startTime"] = summary.map(lambda s: clean_time(s.get("startTime")))
df_activity["endTime"] = summary.map(lambda s: clean_time(s.get("endTime")))
df_activity["stepCount"] = summary.map(lambda s: s.get("stepCount"))
df_activity["stepsDistance"] = summary.map(lambda s: s.get("stepsDistance"))
df_activity["calories"] = summary.map(lambda s: s.get("calories"))
df_activity["sleepQuality"] = summary.map(lambda s: s.get("sleepQuality"))
df_activity["sleepDuration"] = summary.map(lambda s: pt_to_hms(s.get("sleepDuration")))
df_activity["inactivityAlertCount"] = summary.map(lambda s: s.get("inactivityAlertCount"))
df_activity["dailyMetMinutes"] = summary.map(lambda s: s.get("dailyMetMinutes"))

print(df_activity[["startTime","endTime","stepCount","stepsDistance","calories","sleepQuality","sleepDuration","inactivityAlertCount","dailyMetMinutes"]].head())

  startTime   endTime  stepCount  stepsDistance  calories  sleepQuality  \
0  00:00:00  23:59:59    14080.0    9755.738281    2547.0      0.872167   
1      None      None        NaN            NaN       NaN           NaN   
2  00:00:00  23:59:59    25568.0   21580.402344    3181.0      0.000000   
3  00:00:00  23:59:59    11060.0    6132.661621    2651.0      0.885745   
4  00:00:00  23:59:59    27691.0   21221.250000    3294.0      0.850627   

  sleepDuration  inactivityAlertCount  dailyMetMinutes  
0      09:11:30                   0.0           367.00  
1          None                   NaN              NaN  
2      00:00:00                   0.0           915.30  
3      07:39:30                   0.0           271.45  
4      07:18:30                   0.0           776.70  


In [ ]:
# Alle activityLevels aus df_activity['summary'] extrahieren
summary = df_activity["summary"].apply(parse_summary)

# 9 Levelnamen aus den Daten automatisch sammeln
level_names = sorted({
    lvl["level"]
    for s in summary
    for lvl in s.get("activityLevels", [])
    if isinstance(lvl, dict) and "level" in lvl
})

# Neue Spalten hinzufügen (HH:MM:SS)
for lvl in level_names:
    df_activity[lvl] = summary.map(
        lambda s: pt_to_hms(
            next(
                (a["duration"] for a in s.get("activityLevels", []) if a.get("level") == lvl),
                None
            )
        )
    )

print("Neue Spalten:", level_names)
print(df_activity[level_names].head(3))

Neue Spalten: ['CONTINUOS_MODERATE', 'CONTINUOS_VIGOROUS', 'INTERMITTENT_MODERATE', 'INTERMITTENT_VIGOROUS', 'LIGHT', 'NON_WEAR', 'NO_DATA', 'SEDENTARY', 'SLEEP']
  CONTINUOS_MODERATE CONTINUOS_VIGOROUS INTERMITTENT_MODERATE  \
0           01:10:00           00:42:00              00:00:00   
1               None               None                  None   
2           00:58:00           02:37:30              00:00:00   

  INTERMITTENT_VIGOROUS     LIGHT  NON_WEAR   NO_DATA SEDENTARY     SLEEP  
0              00:00:00  04:24:30  02:54:30  00:00:00  07:02:30  07:46:30  
1                  None      None      None      None      None      None  
2              00:00:00  04:30:30  11:06:30  00:00:00  02:49:00  01:58:30  


In [94]:
df_activity = df_activity.drop('summary', axis=1)
df_activity.head()

,date,samples,physicalInformation,startTime,endTime,stepCount,stepsDistance,calories,sleepQuality,sleepDuration,...,dailyMetMinutes,CONTINUOS_MODERATE,CONTINUOS_VIGOROUS,INTERMITTENT_MODERATE,INTERMITTENT_VIGOROUS,LIGHT,NON_WEAR,NO_DATA,SEDENTARY,SLEEP
0,2024-01-26,"{'mets': [{'localTime': '00:00:00.000', 'value...","{'sex': 'FEMALE', 'birthday': '1995-04-19', 'h...",00:00:00,23:59:59,14080.0,9755.738281,2547.0,0.872167,09:11:30,...,367.00,01:10:00,00:42:00,00:00:00,00:00:00,04:24:30,02:54:30,00:00:00,07:02:30,07:46:30
1,2015-11-27,"{'mets': [{'value': 0.875}, {'value': 0.875}, ...","{'sex': 'FEMALE', 'birthday': '1995-04-19', 'h...",None,None,NaN,NaN,NaN,NaN,None,...,NaN,None,None,None,None,None,None,None,None,None
2,2025-06-21,"{'mets': [{'localTime': '00:00:00.000', 'value...","{'sex': 'FEMALE', 'birthday': '1995-04-19', 'h...",00:00:00,23:59:59,25568.0,21580.402344,3181.0,0.000000,00:00:00,...,915.30,00:58:00,02:37:30,00:00:00,00:00:00,04:30:30,11:06:30,00:00:00,02:49:00,01:58:30
3,2021-09-16,"{'mets': [{'localTime': '00:00:00.000', 'value...","{'sex': 'FEMALE', 'birthday': '1995-04-19', 'h...",00:00:00,23:59:59,11060.0,6132.661621,2651.0,0.885745,07:39:30,...,271.45,01:57:30,00:01:00,00:00:00,00:00:00,08:10:30,00:08:30,00:00:00,06:27:00,07:15:30
4,2024-07-15,"{'mets': [{'localTime': '00:00:00.000', 'value...","{'sex': 'FEMALE', 'birthday': '1995-04-19', 'h...",00:00:00,23:59:59,27691.0,21221.250000,3294.0,0.850627,07:18:30,...,776.70,01:19:30,02:12:30,00:00:00,00:00:00,04:55:30,00:00:00,00:00:00,07:42:00,07:50:30


In [96]:
df_activity['samples'][1]
df_activity.drop('samples', axis=1)

,date,physicalInformation,startTime,endTime,stepCount,stepsDistance,calories,sleepQuality,sleepDuration,inactivityAlertCount,dailyMetMinutes,CONTINUOS_MODERATE,CONTINUOS_VIGOROUS,INTERMITTENT_MODERATE,INTERMITTENT_VIGOROUS,LIGHT,NON_WEAR,NO_DATA,SEDENTARY,SLEEP
0,2024-01-26,"{'sex': 'FEMALE', 'birthday': '1995-04-19', 'h...",00:00:00,23:59:59,14080.0,9755.738281,2547.0,0.872167,09:11:30,0.0,367.00,01:10:00,00:42:00,00:00:00,00:00:00,04:24:30,02:54:30,00:00:00,07:02:30,07:46:30
1,2015-11-27,"{'sex': 'FEMALE', 'birthday': '1995-04-19', 'h...",None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,None,None,None,None,None,None,None
2,2025-06-21,"{'sex': 'FEMALE', 'birthday': '1995-04-19', 'h...",00:00:00,23:59:59,25568.0,21580.402344,3181.0,0.000000,00:00:00,0.0,915.30,00:58:00,02:37:30,00:00:00,00:00:00,04:30:30,11:06:30,00:00:00,02:49:00,01:58:30
3,2021-09-16,"{'sex': 'FEMALE', 'birthday': '1995-04-19', 'h...",00:00:00,23:59:59,11060.0,6132.661621,2651.0,0.885745,07:39:30,0.0,271.45,01:57:30,00:01:00,00:00:00,00:00:00,08:10:30,00:08:30,00:00:00,06:27:00,07:15:30
4,2024-07-15,"{'sex': 'FEMALE', 'birthday': '1995-04-19', 'h...",00:00:00,23:59:59,27691.0,21221.250000,3294.0,0.850627,07:18:30,0.0,776.70,01:19:30,02:12:30,00:00:00,00:00:00,04:55:30,00:00:00,00:00:00,07:42:00,07:50:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3082,2019-05-31,"{'sex': 'FEMALE', 'birthday': '1995-04-19', 'h...",00:00:00,23:59:59,0.0,0.000000,1254.0,0.000000,00:00:00,0.0,0.00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,24:00:00,00:00:00,00:00:00,00:00:00
3083,2025-08-12,"{'sex': 'FEMALE', 'birthday': '1995-04-19', 'h...",00:00:00,23:59:59,9862.0,4984.662109,2386.0,0.844280,07:52:00,0.0,216.15,01:02:00,00:15:00,00:00:00,00:00:00,07:31:00,00:27:00,00:00:00,05:59:30,08:45:30
3084,2017-11-04,"{'sex': 'FEMALE', 'birthday': '1995-04-19', 'h...",None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,None,None,None,None,None,None,None
3085,2021-06-17,"{'sex': 'FEMALE', 'birthday': '1995-04-19', 'h...",00:00:00,23:59:59,16060.0,10128.861328,2596.0,0.906209,06:18:30,0.0,333.25,01:58:30,00:18:30,00:00:00,00:00:00,06:08:30,02:26:30,00:00:00,04:41:00,08:27:00


In [99]:
df_activity['physicalInformation'][4]


{'sex': 'FEMALE',
 'birthday': '1995-04-19',
 'height, cm': 168.0,
 'weight, kg': 64.0,
 'vo2Max': 44,
 'maximumHeartRate': 197,
 'sleepGoal': 'PT28800S'}

In [100]:
def pt_to_hms(s):
    if not isinstance(s, str) or not s.startswith("PT"):
        return None
    secs = int(float(s[2:-1]))
    h, rem = divmod(secs, 3600)
    m, s = divmod(rem, 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

info = df_activity["physicalInformation"].apply(parse_summary)

df_activity["vo2Max"] = info.map(lambda d: d.get("vo2Max"))
df_activity["maximumHeartRate"] = info.map(lambda d: d.get("maximumHeartRate"))
df_activity["sleepGoal"] = info.map(lambda d: pt_to_hms(d.get("sleepGoal")))

print(df_activity[["vo2Max", "maximumHeartRate", "sleepGoal"]].head())

   vo2Max  maximumHeartRate sleepGoal
0    44.0             197.0  08:00:00
1     NaN               NaN      None
2    44.0             197.0  08:00:00
3     NaN               NaN  08:00:00
4    44.0             197.0  08:00:00


In [105]:
df_activity = df_activity.drop('samples', axis=1)
df_activity = df_activity.drop('physicalInformation', axis=1)
df_activity.head()

,date,startTime,endTime,stepCount,stepsDistance,calories,sleepQuality,sleepDuration,inactivityAlertCount,dailyMetMinutes,...,INTERMITTENT_MODERATE,INTERMITTENT_VIGOROUS,LIGHT,NON_WEAR,NO_DATA,SEDENTARY,SLEEP,vo2Max,maximumHeartRate,sleepGoal
0,2024-01-26,00:00:00,23:59:59,14080.0,9755.738281,2547.0,0.872167,09:11:30,0.0,367.00,...,00:00:00,00:00:00,04:24:30,02:54:30,00:00:00,07:02:30,07:46:30,44.0,197.0,08:00:00
1,2015-11-27,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,...,None,None,None,None,None,None,None,NaN,NaN,None
2,2025-06-21,00:00:00,23:59:59,25568.0,21580.402344,3181.0,0.000000,00:00:00,0.0,915.30,...,00:00:00,00:00:00,04:30:30,11:06:30,00:00:00,02:49:00,01:58:30,44.0,197.0,08:00:00
3,2021-09-16,00:00:00,23:59:59,11060.0,6132.661621,2651.0,0.885745,07:39:30,0.0,271.45,...,00:00:00,00:00:00,08:10:30,00:08:30,00:00:00,06:27:00,07:15:30,NaN,NaN,08:00:00
4,2024-07-15,00:00:00,23:59:59,27691.0,21221.250000,3294.0,0.850627,07:18:30,0.0,776.70,...,00:00:00,00:00:00,04:55:30,00:00:00,00:00:00,07:42:00,07:50:30,44.0,197.0,08:00:00


In [106]:
df_activity.to_csv("activity_clean.csv", index=False)

# Trainings Daten extrahieren

In [35]:
training_files = [f for f in data_dir.glob("*.json") if f.name.startswith("training-session-")]
print(f"{len(training_files)} Trainings-Dateien gefunden")

training_rows = []
for file in training_files:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
    parts = file.name.split("-")
    date_str = "-".join(parts[2:5])  # 2017-07-23
    training_rows.append({"filename": file.name, "date": date_str, **data})

df_training = pd.DataFrame(training_rows)
df_training = df_training.drop(columns=['filename', 'exportVersion', 'deviceId', 'latitude', 'longitude', 'note', 'feeling'])
df_training["date"] = pd.to_datetime(df_training["date"], errors="coerce")
print(df_training.head(3))


780 Trainings-Dateien gefunden
        date    name                startTime                 stopTime  \
0 2017-07-23  Laufen  2017-07-23T14:53:26.000  2017-07-23T15:58:59.000   
1 2024-02-12          2024-02-12T20:14:26.218  2024-02-12T20:44:02.315   
2 2023-03-19          2023-03-19T17:40:45.489  2023-03-19T19:12:37.569   

   timeZoneOffset  distance     duration  kiloCalories  \
0             120   10503.0  PT3931.875S         587.0   
1              60       NaN  PT1692.965S         206.0   
2              60       NaN  PT5452.219S         732.0   

                         physicalInformationSnapshot  \
0  {'dateTime': '2017-07-23T14:18:08.000', 'sex':...   
1  {'dateTime': '2024-02-12T19:45:38.000', 'sex':...   
2  {'dateTime': '2023-03-19T18:13:29.000', 'sex':...   

                                           exercises  maximumHeartRate  \
0  [{'startTime': '2017-07-23T14:53:26.000', 'sto...               NaN   
1  [{'startTime': '2024-02-12T20:14:26.320', 'sto...             1

In [36]:
df_training.columns

Index(['date', 'name', 'startTime', 'stopTime', 'timeZoneOffset', 'distance',
       'duration', 'kiloCalories', 'physicalInformationSnapshot', 'exercises',
       'maximumHeartRate', 'averageHeartRate', 'periodData',
       'loadInformation'],
      dtype='object')

In [37]:
df_training["startTime"] = pd.to_datetime(df_training["startTime"], errors="coerce").dt.strftime("%H:%M:%S")
df_training["stopTime"]  = pd.to_datetime(df_training["stopTime"], errors="coerce").dt.strftime("%H:%M:%S")
df_training.head()



,date,name,startTime,stopTime,timeZoneOffset,distance,duration,kiloCalories,physicalInformationSnapshot,exercises,maximumHeartRate,averageHeartRate,periodData,loadInformation
0,2017-07-23,Laufen,14:53:26,15:58:59,120,10503.000000,PT3931.875S,587.0,"{'dateTime': '2017-07-23T14:18:08.000', 'sex':...","[{'startTime': '2017-07-23T14:53:26.000', 'sto...",NaN,NaN,NaN,NaN
1,2024-02-12,,20:14:26,20:44:02,60,NaN,PT1692.965S,206.0,"{'dateTime': '2024-02-12T19:45:38.000', 'sex':...","[{'startTime': '2024-02-12T20:14:26.320', 'sto...",137.0,128.0,"{'type': 'TRAINING_SESSION', 'start': '2024-02...","{'cardioLoad': 29.5534, 'muscleLoad': -1.0, 'c..."
2,2023-03-19,,17:40:45,19:12:37,60,NaN,PT5452.219S,732.0,"{'dateTime': '2023-03-19T18:13:29.000', 'sex':...","[{'startTime': '2023-03-19T17:40:45.670', 'sto...",153.0,136.0,"{'type': 'TRAINING_SESSION', 'start': '2023-03...","{'cardioLoad': 118.657, 'muscleLoad': -1.0, 'c..."
3,2023-04-29,,08:14:07,11:13:54,120,72330.203125,PT10474.297S,1680.0,"{'dateTime': '2023-04-29T10:05:59.000', 'sex':...","[{'startTime': '2023-04-29T08:14:08.062', 'sto...",179.0,160.0,"{'type': 'TRAINING_SESSION', 'start': '2023-04...","{'cardioLoad': 387.229, 'muscleLoad': -1.0, 'c..."
4,2023-05-22,,17:46:14,18:07:53,120,770.400024,PT1040.341S,143.0,"{'dateTime': '2023-05-22T16:09:48.000', 'sex':...","[{'startTime': '2023-05-22T17:46:14.761', 'sto...",161.0,142.0,"{'type': 'TRAINING_SESSION', 'start': '2023-05...","{'cardioLoad': 26.2706, 'muscleLoad': -1.0, 'c..."


In [43]:
# Differenz als timedelta berechnen
start_dt = pd.to_datetime(df_training["startTime"], format="%H:%M:%S", errors="coerce")
stop_dt  = pd.to_datetime(df_training["stopTime"], format="%H:%M:%S", errors="coerce")

df_training["duration"] = (stop_dt - start_dt).dt.components.apply(
    lambda x: f"{int(x.hours):02d}:{int(x.minutes):02d}:{int(x.seconds):02d}", axis=1
)

In [45]:
df_training.drop('duration_minutes', axis=1)
df_training.head()

,date,name,startTime,stopTime,timeZoneOffset,distance,duration,kiloCalories,physicalInformationSnapshot,exercises,maximumHeartRate,averageHeartRate,periodData,loadInformation,duration_minutes
0,2017-07-23,Laufen,14:53:26,15:58:59,120,10503.000000,01:05:33,587.0,"{'dateTime': '2017-07-23T14:18:08.000', 'sex':...","[{'startTime': '2017-07-23T14:53:26.000', 'sto...",NaN,NaN,NaN,NaN,01:05:33
1,2024-02-12,,20:14:26,20:44:02,60,NaN,00:29:36,206.0,"{'dateTime': '2024-02-12T19:45:38.000', 'sex':...","[{'startTime': '2024-02-12T20:14:26.320', 'sto...",137.0,128.0,"{'type': 'TRAINING_SESSION', 'start': '2024-02...","{'cardioLoad': 29.5534, 'muscleLoad': -1.0, 'c...",00:29:36
2,2023-03-19,,17:40:45,19:12:37,60,NaN,01:31:52,732.0,"{'dateTime': '2023-03-19T18:13:29.000', 'sex':...","[{'startTime': '2023-03-19T17:40:45.670', 'sto...",153.0,136.0,"{'type': 'TRAINING_SESSION', 'start': '2023-03...","{'cardioLoad': 118.657, 'muscleLoad': -1.0, 'c...",01:31:52
3,2023-04-29,,08:14:07,11:13:54,120,72330.203125,02:59:47,1680.0,"{'dateTime': '2023-04-29T10:05:59.000', 'sex':...","[{'startTime': '2023-04-29T08:14:08.062', 'sto...",179.0,160.0,"{'type': 'TRAINING_SESSION', 'start': '2023-04...","{'cardioLoad': 387.229, 'muscleLoad': -1.0, 'c...",02:59:47
4,2023-05-22,,17:46:14,18:07:53,120,770.400024,00:21:39,143.0,"{'dateTime': '2023-05-22T16:09:48.000', 'sex':...","[{'startTime': '2023-05-22T17:46:14.761', 'sto...",161.0,142.0,"{'type': 'TRAINING_SESSION', 'start': '2023-05...","{'cardioLoad': 26.2706, 'muscleLoad': -1.0, 'c...",00:21:39


In [49]:
df_training['physicalInformationSnapshot'][2]

{'dateTime': '2023-03-19T18:13:29.000',
 'sex': 'FEMALE',
 'birthday': '1995-04-19',
 'height, cm': 168.0,
 'weight, kg': 64.0,
 'vo2Max': 45,
 'maximumHeartRate': 197,
 'restingHeartRate': 55,
 'aerobicThreshold': 148,
 'anaerobicThreshold': 177,
 'sleepGoal': 'PT28800S',
 'functionalThresholdPower': 177}

In [47]:
import ast
import pandas as pd

# sicherstellen, dass die Spalte als String interpretiert wird
def parse_snapshot(val):
    if pd.isna(val):
        return {}
    if isinstance(val, dict):
        return val
    try:
        return ast.literal_eval(val)
    except Exception:
        return {}

# JSON-artige Strings in Dictionaries umwandeln
parsed = df_training["physicalInformationSnapshot"].apply(parse_snapshot)

# gewünschte Werte extrahieren
df_training["vo2Max"] = parsed.apply(lambda x: x.get("vo2Max"))
df_training["maximumHeartRate"] = parsed.apply(lambda x: x.get("maximumHeartRate"))
df_training["restingHeartRate"] = parsed.apply(lambda x: x.get("restingHeartRate"))
df_training["anaerobicThreshold"] = parsed.apply(lambda x: x.get("anaerobicThreshold"))
df_training["functionalThresholdPower"] = parsed.apply(lambda x: x.get("functionalThresholdPower"))

# Sleep Goal (z. B. 'PT28800S' → Sekunden in Stunden umwandeln)
def parse_sleep_goal(x):
    if not isinstance(x, str) or not x.startswith("PT"):
        return None
    seconds = x.replace("PT", "").replace("S", "")
    try:
        return int(seconds) / 3600  # Stunden
    except ValueError:
        return None

df_training["sleepGoal"] = parsed.apply(lambda x: parse_sleep_goal(x.get("sleepGoal")))

In [ ]:
df_training = df_training.drop('physicalInformationSnapshot', axis=1)
df_training = df_training.drop('periodData', axis=1)
df_training = df_training.drop('loadInformation', axis=1)

,date,name,startTime,stopTime,timeZoneOffset,distance,duration,kiloCalories,exercises,maximumHeartRate,averageHeartRate,periodData,loadInformation,duration_minutes,vo2Max,restingHeartRate,anaerobicThreshold,functionalThresholdPower,sleepGoal
0,2017-07-23,Laufen,14:53:26,15:58:59,120,10503.000000,01:05:33,587.0,"[{'startTime': '2017-07-23T14:53:26.000', 'sto...",198,NaN,NaN,NaN,01:05:33,48,55.0,178.0,NaN,NaN
1,2024-02-12,,20:14:26,20:44:02,60,NaN,00:29:36,206.0,"[{'startTime': '2024-02-12T20:14:26.320', 'sto...",197,128.0,"{'type': 'TRAINING_SESSION', 'start': '2024-02...","{'cardioLoad': 29.5534, 'muscleLoad': -1.0, 'c...",00:29:36,44,55.0,177.0,172.0,8.0
2,2023-03-19,,17:40:45,19:12:37,60,NaN,01:31:52,732.0,"[{'startTime': '2023-03-19T17:40:45.670', 'sto...",197,136.0,"{'type': 'TRAINING_SESSION', 'start': '2023-03...","{'cardioLoad': 118.657, 'muscleLoad': -1.0, 'c...",01:31:52,45,55.0,177.0,177.0,8.0
3,2023-04-29,,08:14:07,11:13:54,120,72330.203125,02:59:47,1680.0,"[{'startTime': '2023-04-29T08:14:08.062', 'sto...",197,160.0,"{'type': 'TRAINING_SESSION', 'start': '2023-04...","{'cardioLoad': 387.229, 'muscleLoad': -1.0, 'c...",02:59:47,44,55.0,177.0,172.0,8.0
4,2023-05-22,,17:46:14,18:07:53,120,770.400024,00:21:39,143.0,"[{'startTime': '2023-05-22T17:46:14.761', 'sto...",197,142.0,"{'type': 'TRAINING_SESSION', 'start': '2023-05...","{'cardioLoad': 26.2706, 'muscleLoad': -1.0, 'c...",00:21:39,44,55.0,177.0,172.0,8.0


In [57]:
df_training.head()


,date,name,startTime,stopTime,timeZoneOffset,distance,duration,kiloCalories,exercises,maximumHeartRate,averageHeartRate,duration_minutes,vo2Max,restingHeartRate,anaerobicThreshold,functionalThresholdPower,sleepGoal
0,2017-07-23,Laufen,14:53:26,15:58:59,120,10503.000000,01:05:33,587.0,"[{'startTime': '2017-07-23T14:53:26.000', 'sto...",198,NaN,01:05:33,48,55.0,178.0,NaN,NaN
1,2024-02-12,,20:14:26,20:44:02,60,NaN,00:29:36,206.0,"[{'startTime': '2024-02-12T20:14:26.320', 'sto...",197,128.0,00:29:36,44,55.0,177.0,172.0,8.0
2,2023-03-19,,17:40:45,19:12:37,60,NaN,01:31:52,732.0,"[{'startTime': '2023-03-19T17:40:45.670', 'sto...",197,136.0,01:31:52,45,55.0,177.0,177.0,8.0
3,2023-04-29,,08:14:07,11:13:54,120,72330.203125,02:59:47,1680.0,"[{'startTime': '2023-04-29T08:14:08.062', 'sto...",197,160.0,02:59:47,44,55.0,177.0,172.0,8.0
4,2023-05-22,,17:46:14,18:07:53,120,770.400024,00:21:39,143.0,"[{'startTime': '2023-05-22T17:46:14.761', 'sto...",197,142.0,00:21:39,44,55.0,177.0,172.0,8.0


In [58]:
df_training['exercises'][2]

[{'startTime': '2023-03-19T17:40:45.670',
  'stopTime': '2023-03-19T19:11:37.889',
  'timezoneOffset': 60,
  'duration': 'PT5452.219S',
  'sport': 'INDOOR_CYCLING',
  'kiloCalories': 732,
  'heartRate': {'min': 94, 'avg': 136, 'max': 153},
  'zones': {'heart_rate': [{'lowerLimit': 99,
     'higherLimit': 118,
     'inZone': 'PT364S',
     'zoneIndex': 1},
    {'lowerLimit': 118,
     'higherLimit': 138,
     'inZone': 'PT2073S',
     'zoneIndex': 2},
    {'lowerLimit': 138,
     'higherLimit': 158,
     'inZone': 'PT3005S',
     'zoneIndex': 3},
    {'lowerLimit': 158, 'higherLimit': 177, 'inZone': 'PT0S', 'zoneIndex': 4},
    {'lowerLimit': 177,
     'higherLimit': 197,
     'inZone': 'PT0S',
     'zoneIndex': 5}]},
  'samples': {'heartRate': [{'dateTime': '2023-03-19T17:40:45.670',
     'value': 99},
    {'dateTime': '2023-03-19T17:40:46.670', 'value': 97},
    {'dateTime': '2023-03-19T17:40:47.670', 'value': 96},
    {'dateTime': '2023-03-19T17:40:48.670', 'value': 95},
    {'dateTi

In [ ]:
import ast
import pandas as pd

# sicherstellen, dass die Spalte als String interpretiert wird
def parse_exercises(val):
    if isinstance(val, str):
        try:
            val = ast.literal_eval(val)
        except Exception:
            return []
    if isinstance(val, list):
        return val
    return []

# JSON-artige Strings in Python-Objekte umwandeln
parsed = df_training["exercises"].apply(parse_exercises)

# neue Spalten erstellen
df_training["sport"] = parsed.apply(lambda x: x[0].get("sport") if len(x) > 0 else None)
df_training["minHeartRate"] = parsed.apply(lambda x: x[0].get("heartRate", {}).get("min") if len(x) > 0 else None)

                 sport  minHeartRate
0              RUNNING           NaN
1       INDOOR_CYCLING         100.0
2       INDOOR_CYCLING          94.0
3          ROAD_BIKING          89.0
4  OPEN_WATER_SWIMMING          90.0


In [65]:
df_training['sport'].value_counts()
df_training = df_training.drop('exercises', axis=1)
df_training.head()


,date,name,startTime,stopTime,timeZoneOffset,distance,duration,kiloCalories,maximumHeartRate,averageHeartRate,duration_minutes,vo2Max,restingHeartRate,anaerobicThreshold,functionalThresholdPower,sleepGoal,sport,minHeartRate
0,2017-07-23,Laufen,14:53:26,15:58:59,120,10503.000000,01:05:33,587.0,198,NaN,01:05:33,48,55.0,178.0,NaN,NaN,RUNNING,NaN
1,2024-02-12,,20:14:26,20:44:02,60,NaN,00:29:36,206.0,197,128.0,00:29:36,44,55.0,177.0,172.0,8.0,INDOOR_CYCLING,100.0
2,2023-03-19,,17:40:45,19:12:37,60,NaN,01:31:52,732.0,197,136.0,01:31:52,45,55.0,177.0,177.0,8.0,INDOOR_CYCLING,94.0
3,2023-04-29,,08:14:07,11:13:54,120,72330.203125,02:59:47,1680.0,197,160.0,02:59:47,44,55.0,177.0,172.0,8.0,ROAD_BIKING,89.0
4,2023-05-22,,17:46:14,18:07:53,120,770.400024,00:21:39,143.0,197,142.0,00:21:39,44,55.0,177.0,172.0,8.0,OPEN_WATER_SWIMMING,90.0


In [66]:
df_training.to_csv("training_clean.csv", index=False)

# 24/7 Daten extrahieren

In [107]:
# Alle 247ohr_*.json Dateien finden
ohr_files = [f for f in data_dir.glob("*.json") if f.name.startswith("247ohr_")]
print(f"{len(ohr_files)} 247ohr_-Dateien gefunden")

ohr_rows = []
for file in ohr_files:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)

    date_part = file.name.split("-")[0].replace("247ohr_", "")  # z.B. "2021_10"
    ohr_rows.append({"filename": file.name, "date_raw": date_part, **data})

df_247 = pd.DataFrame(ohr_rows)

# Datum konvertieren
if not df_247.empty:
    df_247["date"] = pd.to_datetime(df_247["date_raw"], format="%Y_%m", errors="coerce")

print(df_247.head(3))

55 247ohr_-Dateien gefunden
                                            filename date_raw  \
0  247ohr_2021_10-57ac3fcc-e24a-4d38-be68-fc7afed...  2021_10   
1  247ohr_2022_05-1b4dc29b-5128-4809-a903-dca5ef1...  2022_05   
2  247ohr_2021_05-76192525-61c7-4c6b-b7ec-7b147f3...  2021_05   

                                          deviceDays       date  
0  [{'userId': 10995975, 'deviceId': '87FE0E2E', ... 2021-10-01  
1  [{'userId': 10995975, 'deviceId': '87FE0E2E', ... 2022-05-01  
2  [{'userId': 10995975, 'deviceId': '87FE0E2E', ... 2021-05-01  


In [108]:
df_247.columns

Index(['filename', 'date_raw', 'deviceDays', 'date'], dtype='object')

In [111]:
df_247.head()

,filename,date_raw,deviceDays,date
0,247ohr_2021_10-57ac3fcc-e24a-4d38-be68-fc7afed...,2021_10,"[{'userId': 10995975, 'deviceId': '87FE0E2E', ...",2021-10-01
1,247ohr_2022_05-1b4dc29b-5128-4809-a903-dca5ef1...,2022_05,"[{'userId': 10995975, 'deviceId': '87FE0E2E', ...",2022-05-01
2,247ohr_2021_05-76192525-61c7-4c6b-b7ec-7b147f3...,2021_05,"[{'userId': 10995975, 'deviceId': '87FE0E2E', ...",2021-05-01
3,247ohr_2024_09-df912a4b-b0da-49b1-935c-154b820...,2024_09,"[{'userId': 10995975, 'deviceId': '87FE0E2E', ...",2024-09-01
4,247ohr_2021_09-b092ad1c-83a5-4eea-8d06-d39e34d...,2021_09,"[{'userId': 10995975, 'deviceId': '87FE0E2E', ...",2021-09-01


In [115]:
df_247['date_raw']
df_247 = df_247.drop('date_raw', axis=1)
df_247 = df_247.drop('filename', axis=1)

In [117]:
df_247.head()

,deviceDays,date
0,"[{'userId': 10995975, 'deviceId': '87FE0E2E', ...",2021-10-01
1,"[{'userId': 10995975, 'deviceId': '87FE0E2E', ...",2022-05-01
2,"[{'userId': 10995975, 'deviceId': '87FE0E2E', ...",2021-05-01
3,"[{'userId': 10995975, 'deviceId': '87FE0E2E', ...",2024-09-01
4,"[{'userId': 10995975, 'deviceId': '87FE0E2E', ...",2021-09-01


In [113]:
df_247['deviceDays'][2]

[{'userId': 10995975,
  'deviceId': '87FE0E2E',
  'date': '2021-05-01',
  'samples': [{'heartRate': 56,
    'secondsFromDayStart': 251,
    'source': 'TIMED_24_7'},
   {'heartRate': 56, 'secondsFromDayStart': 551, 'source': 'TIMED_24_7'},
   {'heartRate': 56, 'secondsFromDayStart': 851, 'source': 'TIMED_24_7'},
   {'heartRate': 55, 'secondsFromDayStart': 1151, 'source': 'TIMED_24_7'},
   {'heartRate': 56, 'secondsFromDayStart': 1452, 'source': 'TIMED_24_7'},
   {'heartRate': 56, 'secondsFromDayStart': 1752, 'source': 'TIMED_24_7'},
   {'heartRate': 56, 'secondsFromDayStart': 2052, 'source': 'TIMED_24_7'},
   {'heartRate': 57, 'secondsFromDayStart': 2352, 'source': 'TIMED_24_7'},
   {'heartRate': 58, 'secondsFromDayStart': 2652, 'source': 'TIMED_24_7'},
   {'heartRate': 58, 'secondsFromDayStart': 2951, 'source': 'TIMED_24_7'},
   {'heartRate': 59, 'secondsFromDayStart': 3252, 'source': 'TIMED_24_7'},
   {'heartRate': 59, 'secondsFromDayStart': 3553, 'source': 'TIMED_24_7'},
   {'heartRa

In [ ]:
import ast
import pandas as pd

# deviceDays sauber in Listen umwandeln
def parse_list(v):
    if isinstance(v, list):
        return v
    if isinstance(v, str):
        try:
            return ast.literal_eval(v)
        except Exception:
            return []
    return []

# alle Tagesdatensätze sammeln
rows = []
for _, row in df_247.iterrows():
    for day in parse_list(row["deviceDays"]):
        date = day.get("date")
        for sample in day.get("samples", []):
            rows.append({
                "date": date,
                "time": pd.to_datetime(sample.get("secondsFromDayStart"), unit="s").strftime("%H:%M:%S"),
                "heartRate": sample.get("heartRate"),
            })

df_hr = pd.DataFrame(rows)
df_hr["date"] = pd.to_datetime(df_hr["date"])

print(df_hr.head())

        date      time  heartRate  source
0 2021-10-07  16:32:56         89  MANUAL
1 2021-10-07  16:32:57         90  MANUAL
2 2021-10-07  16:32:58         91  MANUAL
3 2021-10-07  16:32:59         92  MANUAL
4 2021-10-07  16:33:00         92  MANUAL


In [121]:
df_hr.shape

(7095860, 4)

In [122]:
df_hr.head()

,date,time,heartRate,source
0,2021-10-07,16:32:56,89,MANUAL
1,2021-10-07,16:32:57,90,MANUAL
2,2021-10-07,16:32:58,91,MANUAL
3,2021-10-07,16:32:59,92,MANUAL
4,2021-10-07,16:33:00,92,MANUAL


In [123]:
df_hr.drop('source', axis=1)

,date,time,heartRate
0,2021-10-07,16:32:56,89
1,2021-10-07,16:32:57,90
2,2021-10-07,16:32:58,91
3,2021-10-07,16:32:59,92
4,2021-10-07,16:33:00,92
...,...,...,...
7095855,2024-10-31,23:39:03,67
7095856,2024-10-31,23:44:04,68
7095857,2024-10-31,23:49:04,69
7095858,2024-10-31,23:54:04,69


In [124]:
df_hr.to_csv("df_247_clean.csv", index=False)